### __Guidelines for using this notebook__
#### The notebook generates connectivity matrices used as input for the analytical framework (*gcolab_find_en.ipynb*).
###### Input for this notebook is an EDF file - a seizure recording of at least 30 s of pre-seizure time and the whole seizure
###### EDF filenames are: *subid-runid.EDF*
###### The seizure file has been reformatted before from the Brainvision format to EDF using MNE module. The seizure start and end annotations are imported from the metadata Excel file created before, and are later defined as variables: *sz_start_note*, *sz_end_note*.
###### The connectivity matrices will be computed for 1-s epochs within two time intervals - 30-s preseizure interval and 30-s seizure interval, starting at seizure start.
###### Once the notebook is run, the user will be asked to choose from a number of parameters:
###### 1) Connectivity measure
###### 2) If the signal should be filtered in a specific frequency band
###### 3) If yes, define the minimum frequency limit
###### 4) and the maximum frequency limit.
###### The results will be saved as a rec object (defined in data_legacy module), containing the connectivity matrices as '*X*', epoch labels (used as ground truth by the classifier) as '*y*' and node labels as '*nodes*', in a subfolder called '*connectivity_matrices*'.

#### The following five cells define objects for loading and handling the EDF data.  

In [ ]:
from inspect import ismethod
from datetime import timedelta as _time
from datetime import datetime
from collections.abc import Iterable as iterable

def some(field): return (field is not None and field != [] and field != {} and field != ()) or field == True
def no(field): return not some(field) or field==False or field==''

class class_of:
    _instance = None
    def __init__(_, object):
        _._is = type(object)
    def inherits(_, *types):
        return issubclass(_._is, types)
    def has(_, *types): return _.inherits(*types)
    def __enter__(self):
        self._instance = self
        return self
    def __exit__(self, type, value, traceback): self._instance = None
    @staticmethod
    def each_in(list):
        if isiterable(list):
            return [type(item) for item in list]

class struct:
    def __init__(table, **sets): table.__dict__.update(sets)
    @property
    def sets(this): return set(dir(this)) - set(dir(type(this)))
    def set(object, **fields):
        for field in fields: setattr(object, field, fields[field])
    def get(object, *fields): return [getattr(object, field) for field in fields if field in object.__dict__]
    def _clonable(set, mask=None):
        check = set.__dict__.copy()
        clonable = check.copy()
        if some(mask): pass
#            for field in check:
#                if sum([int(_(check[field])) for _ in mask])+sum([int(_(field)) for _ in mask]): clonable.pop(field)
        return clonable
    @staticmethod
    def _from(type):
        if hasattr(type, '__dict__'): return struct(**type.__dict__.copy())
        return struct()

def meta(data, *mask): return struct._from(data)._clonable(mask)
def get(data, *fields):
    if not issubclass(type(data), dict): data=struct._from(data)._clonable()
    return struct(**data).get(*fields)

class table(struct):
    def _default(field, name, value):
        try: return getattr(field, name)
        except: setattr(field, name, value)
        return value
    def clear(this, *fields):
        sets = this.sets
        if not fields: fields = sets
        if fields:
            set = [field for field in fields if hasattr(this,field) and not ismethod(getattr(this, field))]
            for field in set: delattr(this, field)
    def has(this, *fields):
        return all([hasattr(this, field) for field in fields])
    def has_not(this, *fields): return not this.has(*fields)
    def check(this, **KV):
        try: check = [KV[key]==this.__dict__[key] for key in KV]
        except: return False
        return all(check)
    def find(this, _type):
        return [value for value in this.sets if class_of(get(this,value)[0]).inherits(_type)]
    def clone(this):
        clone = type(this)()
        sets = this._clonable()
        clone.set(**sets)
        return clone

def isiterable(this): return isinstance(this, iterable) and type(this) is not str
def default(field, name, value): return table(**field)._default(name, value)

def ni(list):
    if isiterable(list):
        for n,i in enumerate(list): yield n,i
    else:
        for n,i in enumerate(list.__dict__.keys()): yield n,i

class at(table):
    DAY, HOUR, MIN = 86400, 3600, 60
    def __init__(_, dtime=None, **sets):
        _.set(**sets)
        if some(dtime) and issubclass(type(dtime), _time): _._time = dtime
        else:
            d,h,m,s,ms = _._default('d',0), _._default('h',0), _._default('m',0), _._default('s',0), _._default('ms',0)
            if not any([d,h,m,s,ms]): now=datetime.now(); _._time = now-datetime(now.year, now.month, now.day)
            else: _._time = _time(days=d, hours=h, minutes=m, seconds=s, milliseconds=ms)
        _.clear('d','h','m','s','ms')
    def __sub__(_, dtime):
        of=type(dtime); sets=_._clonable()
        if issubclass(of, _time): return at(_._time-dtime, **sets)
        elif issubclass(of, at): sets.update(dtime._clonable()); return at(_._time-dtime._time, **sets)
    def __add__(_, dtime):
        of=type(dtime); sets=_._clonable()
        if issubclass(of, _time): return at(_._time+dtime, **sets)
        elif issubclass(of, at): sets.update(dtime._clonable()); return at(_._time+dtime._time, **sets)
    def __str__(_): return str(_._time)
    @property
    def seconds(_): return _._time.seconds
    @property
    def S(_): return _.seconds
    @property
    def minutes(_): return _._time.seconds/60
    @property
    def M(_): return _.minutes
    @property
    def hours(_): return _.minutes/60
    @property
    def H(_): return _.hours
    @property
    def days(_): return _._time.days
    @property
    def D(_): return _.days
    @staticmethod
    def zero(): return at(_time())

In [ ]:
from inspect import isfunction, ismethod, isgeneratorfunction, isgenerator, isroutine
from inspect import isabstract, isclass, ismodule, istraceback, isframe, iscode, isbuiltin
from inspect import ismethoddescriptor, isdatadescriptor, isgetsetdescriptor, ismemberdescriptor
from inspect import isawaitable, iscoroutinefunction, iscoroutine

from collections.abc import Iterable as iterable

import pickle

def isfx(field): return ismethod(field) or isfunction(field)

class GhostSet:
    """ enhanced interface (ghost) to retrieve class fields """
    def _meta(data): return {k:v for k,v in data.__dict__.items() if not isfx(v)}
    def _at_last(_, sets): pass
    def _set(object, **sets):
        ''' use to fast initialize fields | needed to avoid initialization problems at copy by value '''
        for field in sets: setattr(object, field, sets[field])
        object._at_last(sets)
GSet = GhostSet

def meta(object):
    ''' retrieves clonable object metadata (__dict__) as a copy '''
    if isinstance(object, GSet): return object._meta()
    return {}

class ClonableObjectGhost:
    """ enhanced interface (ghost) for clonable objects """
    def _by_val(_, depth=-1, _layer=0): pass
GCo = ClonableObjectGhost

class ClonableObject(GSet, GCo):
    """ base clonable object """
    def __init__(this, **data): this._set(**data)
    def __call__(_, **options): _._set(**options)
    def _by_val(_, depth=-1, _layer=0):
        copy = type(_)()
        copy._set(**_._meta())
        if depth<0 or depth>_layer:
            for field in copy.__dict__:
                if isinstance(copy.__dict__[field], ClonableObjectGhost):
                    copy.__dict__[field] = copy.__dict__[field]._by_val(depth,_layer+1)
        return copy
COb = ClonableObject

def copy_by_val(object, depth=-1, _layer=0):
    if isinstance(object, GCo): return object._by_val(depth,_layer)
    return object
copy = by_val = vof = copy_by_val

class ComparableGhost:
    """ enhanced interface (ghost) for comparing instances """
    def _compare(a, b):
        if type(a) != type(b): return False
        if a.__dict__ == b.__dict__: return True
        return False
    def __eq__(a, b): return a._compare(b)
GEq = ComparableGhost

class IterableObjectGhost(GSet):
    """ enhanced interface (ghost) for iterables: exposes __dict__,
        therefore Iterable Objects are like lua dictionaries """
    def __contains__(this, key): return key in this.__dict__
    def __iter__(this): return iter(this.__dict__)
    def items(my): return my.__dict__.items()
    def __getitem__(by, field): return by.__dict__[field]
    def __setitem__(by, field, value): by.__dict__[field] = value
    def pop(by, field): return by.__dict__.pop(field)
GIo = IterableObjectGhost

class ReprGhost:
    """ enhanced interface (ghost) for the skeleton method _repr,
        see implementation of Struct for a working example;
        Record __repr__ override uses _lines_ for max lines display """
    _lines_ = 31
    _chars_ = 13
    _msgsz_ = 62
    _ellipsis_ = ' ... '
    def _repr(my, value):
        _type = ''.join(''.join(str(type(value)).split('class ')).split("'"))
        _value = '{}'.format(value)
        if len(_value)>my._chars_:
            show = int(my._chars_/2)
            _value = _value[:show]+my._ellipsis_+_value[-show:]
        return '{} {}'.format(_type, _value)
    def _resize(this, message, at=.7):
        if len(message)>this._msgsz_:
            start = int(at*this._msgsz_)
            end = this._msgsz_-start
            return message[:start]+this._ellipsis_+message[-end:]
        return message
GRe = ReprGhost

def set_repr_to(lines): GRe._lines_ = lines

class Struct(COb, GEq, GIo, GRe):
    """ structured autoprintable object, behaves like a lua dictionary """
    def __repr__(_):
        return '\n'.join(['{}:\t{}'.format(k, _._repr(v)) for k,v in _.items()])
struct = Struct

class RecordableGhost:
    """ enhanced interface (ghost) for type recording,
        see Record for a working example """
    @staticmethod
    def load(filename):
        with open(filename, 'rb') as file: return pickle.load(file)
    def save(data, filename):
        with open(filename, 'wb') as file: pickle.dump(data, file)

GRec = RecordableGhost

class Record(GSet, GCo, GRec, GEq, GRe):
    """ wrapper for any object or value, auto-inspects and provides load/save type structure """
    data = None
    _check = dict(
            isfunction=isfunction, ismethod=ismethod, isgeneratorfunction=isgeneratorfunction, isgenerator=isgenerator, isroutine=isroutine,
            isabstract=isabstract, isclass=isclass, ismodule=ismodule, istraceback=istraceback, isframe=isframe, iscode=iscode, isbuiltin=isbuiltin,
            ismethoddescriptor=ismethoddescriptor, isdatadescriptor=isdatadescriptor, isgetsetdescriptor=isgetsetdescriptor, ismemberdescriptor=ismemberdescriptor,
            isawaitable=isawaitable, iscoroutinefunction=iscoroutinefunction, iscoroutine=iscoroutine
                   )
    def __init__(this, token, **meta):
        this.data = token
        this.__dict__.update({k:v(token) for k,v in this._check.items()})
        super()._set(**meta)
    @property
    def type(_): return type(_.data)
    def inherits(_, *types): return issubclass(_.type, types)
    @property
    def isbaseiterable(_): return _.inherits(tuple, list, dict, set) or _.isgenerator or _.isgeneratorfunction
    @property
    def isiterable(_): return isinstance(_.data, iterable) and _.type is not str
    def _clone_iterable(_):
        if _.inherits(dict): return _.data.copy()
        elif _.isgenerator or _.isgeneratorfunction: return (i for i in list(_.data))
        else: return type(_.data)(list(_.data)[:])
    def _meta(data): return {k:v for k,v in data.__dict__.items() if k != 'data' and not isfx(v)}
    def _by_val(_, depth=-1, layer=0):
        data = _.data
        if _.isiterable: data = _._clone_iterable()
        elif _.inherits(ClonableObjectGhost): data = by_val(data, depth, layer)
        return type(_)(data, **meta(_))
    def __enter__(self): self._instance = self; return self
    def __exit__(self, type, value, traceback): self._instance = None
    def __repr__(self):
        if not hasattr(self, '_preprint'): return Record(self.data, _preprint='', _lines=Record(Record._lines_)).__repr__()
        if self.isbaseiterable:
            pre, repr = self._preprint, ''
            for n,i in enumerate(self.data):
                if self._lines.data == 0: break
                else: self._lines.data -= 1
                index, item = str(n), i
                if self.inherits(dict): index += ' ({})'.format(str(i)); item = self.data[i]
                repr += pre+'{}: '.format(index)
                next = Record(item, _preprint=pre+'\t', _lines=self._lines)
                if next.isiterable: repr += '\n'
                repr += next.__repr__()
                repr += '\n'
            return repr
        elif self.inherits(GCo): return Record(self.data._meta(), _preprint=self._preprint, _lines=self._lines).__repr__()
        else: return self._repr(self.data)
REc = Record

class Bisect(list, COb):
    """ bisect implementation using clonable objects """
    def __init__(set, *items, key=None, reverse=False):
        if not key: key = lambda  x:x
        super().__init__(sorted(items, reverse=reverse, key=key))
    def _bisect(set, item, key, reverse, bottom, top):
        def _(check):
            if key: return key(check)
            return check
        at = int((top-bottom)/2)+bottom
        if len(set)==0: return (0,-1)
        if item==_(set[at]): return (at,0)
        bigger = item<_(set[at])
        if bigger != reverse:
            if at-bottom>0: return set._bisect(item, key, reverse, bottom, at)
            return (at,-1)
        elif top-at>1: return set._bisect(item, key, reverse, at, top)
        return (at,1)
    def search(_, item, key=None, reverse=False):
        if not key: key = lambda x:x
        return _._bisect(item, key, reverse, 0, len(_))
    def _by_val(_, depth=-1, _layer=0):
        copy = super()._by_val(depth, _layer)
        copy += _[:]
        return copy
BSx = Bisect

In [ ]:
from numpy import ndarray, resize, linspace, arange
from numpy import min, max, average, floor
from numpy import ubyte, zeros, array
from scipy.signal import lfilter, butter
from matplotlib import pylab as lab

_NOTCH = _FR = 50
_SAMPLING = 500
_CONTINUOUS = 1
_UNIT = 'ms'

class rec(table, ndarray):
    @property
    def dimensions(of): return len(of.shape)
    @property
    def is_scalar(this): return this.shape is ()
    @property
    def is_vector(this): return len(this.shape)==1
    @property
    def is_matrix(this): return len(this.shape)>1
    @property
    def is_cube(this): return len(this.shape) == 3
    @property
    def is_binary(this): return this.dtype == ubyte and max(this) == 1
    @property
    def serialized(data):
        if not data.is_scalar and data.dimensions>1:
            return rec.read(data.T.flatten(), _deser=data.T.shape, **meta(data))
        return data
    @property
    def deserialized(data):
        if data.has('_deser'):
            deser = rec.read(resize(data, data._deser).T, **meta(data))
            deser.clear('_deser')
            return deser
        return data
    @property
    def as_matrix(data):    #implement numpy matrix
        if data.is_vector: return rec.read([data], to=type(data), **meta(data))
        return data
    @property
    def raw(data):
        if data.shape[0] == 1: return rec.read(data[0], **meta(data)).raw
        return data
    def join(base, *parts, **sets):
        flip, parts = None, list(parts)
        if 'flip' in sets: flip=sets.pop('flip')
        next = parts[0]
        if len(parts)>1: next = rec.join(parts[0], parts[1:])
        congruent = base.dimensions == next.dimensions and base.dimensions < 3
        if congruent:
            sets.update(base._clonable())
            A, B = base, next
            if flip: A, B = base.T, next.T
            C = record(A.tolist()+B.tolist(), **sets)
            if flip: return record(C.T, **sets)
            return C
    def get_as(this, data, cast=None):
        source = this
        if no(cast):
            if issubclass(type(data), rec): cast = type(data)
            else: cast = type(this)
        if issubclass(type(data), ndarray): source = resize(this, data.shape)
        return rec.read(source, to=cast, **meta(data))
    @staticmethod
    def read(iterable, to=None, **sets):
        if no(to) or not issubclass(to, rec): to = rec
        data = array(iterable).view(to)
        data.set(**sets)
        return data
    def clone(this, **sets):
        copy = this.copy().view(type(this))
        sets.update(this._clonable())
        copy.set(**sets)
        return copy
    def exclude(data, *items, **sets):
        if no(items) or data.is_scalar: return data
        excluded, items = None, [item for item in range(len(data)) if item not in items]
        if data.is_vector: excluded = rec.read([data])[:,items][0]
        else: excluded = data[items,:]
        return rec.read(excluded, to=type(data), **meta(data), **sets)
    def include(data, *items, **sets):
        if no(items) or data.is_scalar: return data
        included = []
        if data.is_vector: included = rec.read([data])[:,items][0]
        else: included = data[items,:]
        return rec.read(included, to=type(data), **meta(data), **sets)

create = record = rec.read
line = linspace

def series(ori,end=None,by=1):
    if no(end): end=ori; ori=0
    if not issubclass(type(by), int): return create(arange(ori,end,by))
    return array(range(ori,end,by))

def plot(data, at = 0., spacing = 1., color = 'k', width = 1., offset=0.): #review
    draw = record(data, **meta(data)); at = spacing*draw.as_matrix.shape[0]
    axes = lab.gca(); axes.set_ylim([at+max(data),0-max(data)]); at=0
    for n, row in ni(draw.as_matrix):
        if some(offset): row = draw[n]-average(row)+offset
        c, w = color, width
        if isiterable(color): c = color[n]
        if isiterable(width): w = width[n]
        lab.plot(at+row+n*spacing, color = c, linewidth = w)

def butter_type(lowcut, highcut, fs, order=5, type='band'):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype=type)
    return b, a

def butter_filter(data, lowcut, highcut, fs, order=5, type='band'):
    b, a = butter_type(lowcut, highcut, fs, order=order, type=type)
    y = lfilter(b, a, data)
    return y

def _to_rec(this):
    if not issubclass(type(this), rec): return rec.read(this, **meta(this)), rec
    return this, type(this)

def _prefilt(data, fs):
    pre = []
    for line in data.as_matrix:
        vector = line.tolist()
        pre.append(vector[:int(fs)]+vector)
    return record(pre)

def _postfilt(data, fs):
    post = []
    for line in data:
        vector = line.tolist()
        post.append(vector[int(fs):])
    return post

def notch(this, using=butter_type, fs=_SAMPLING, size=2, at=_NOTCH, order=5):
    data, type = _to_rec(this)
    if data.has('sampling'): fs=data.sampling
    nyq, cutoff = fs / 2., []
    for f in range(int(at), int(nyq), int(at)):
        cutoff.append((f - size, f + size))
    signal = _prefilt(data, fs)
    for bs in cutoff:
        low,hi = bs
        b,a = butter_type(low,hi,fs,order,'bandstop')
        signal = lfilter(b,a,signal)
    return record(_postfilt(signal, fs), to=type, **meta(data))

def band(this, low_high, fs=_SAMPLING, using=butter_filter, order=5):
    data, type = _to_rec(this)
    if data.has('sampling'): fs=data.sampling
    low, high = min(low_high), max(low_high)
    if low<1.: low = 1.
    tailed = _prefilt(data, fs)
    tailed = using(tailed, low, high, fs, order)
    return rec.read(_postfilt(tailed, fs), to=type, **meta(data))

def binarize(this):
    data, type = _to_rec(this)
    if data.is_binary: return data
    rows = []
    for row in data.as_matrix:
        d = row - array([row[-1]]+row[:-1].tolist())
        d[d>=0] = 1; d[d<0] = 0
        rows.append(d.astype(ubyte))
    return rec.read(rows, to=type).get_as(data)

def halve(matrix):
    halved, (data, type) = [], _to_rec(matrix)
    for line in data.as_matrix:
        h = resize(line, (int(len(line)/2), 2))
        halved.append((h[:,0]+h[:,1])/2.)
    return rec.read(halved, to=type, **meta(data))

def dwindle(matrix, by=1):
    if by: return dwindle(halve(matrix), by-1)
    return matrix

def upsample(matrix, fs1, fs2):
    y=zeros((matrix.shape[0],fs2))
    if fs1 < fs2:
        #upsampling by a factor R
        L=matrix.shape[1]
        R=int(floor(fs2/fs1)+1)
        for i,e in enumerate(matrix):
            ups=[]
            for j in range(L-1):
                if j>0: ups.append(list(linspace(matrix[i][j],matrix[i][j+1],R)[1:3]))
                else: ups.append(list(linspace(matrix[i][j],matrix[i][j+1],R)[0:3]))
            for k,s in enumerate(sum(ups, [])): y[i][k]=s
            y[i][-1]=y[i][-2]
        return rec.read(y)
    else: print("Error: fs1 >= fs2")

def remap(this, axis=None, base=0, top=1., e=0):
    def map(x, b, t, e): return ((x-min(x)+e)/(max(x)-min(x)+e)+b)*(t-b)
    data, type = _to_rec(this)
    if no(axis): return rec.read(map(this, base, top, e), to=type, **meta(data))
    rows = data.as_matrix
    if axis==0 or axis>1: rows = rows.T
    remapped = []
    for row in rows: remapped.append(map(row, base, top, e))
    if axis==0 or axis>1: rows = rows.T
    return rec.read(remapped).get_as(data)

In [ ]:
this = Record

from numpy import array, average, median, random
import scipy.stats as stats

class Table(COb, GEq, GRe):
    data = None
    default = None
    PAD = 3
    ELLIPSIS_AT = int(GRe._lines_*.3)
    class _axes(list):
        def insert(_from, this, item):
            super().insert(this, item)
            _from.__dict__[item.name] = item
        def __setitem__(_, pos, axis):
            super().__setitem__(pos, axis)
            _.__dict__[axis.name] = axis
    class axis(list, GSet, GRe):
        name = None
        root = None
        _to = 0
        def __init__(axis, root, labels, name='ax', force_at=None):
            if force_at: root.axes[force_at].name = None
            with this(labels) as dim:
                if not dim.isiterable and dim.inherits(int): labels = range(labels)
            super().__init__(labels)
            names = [ax.name for ax in root.axes]
            name_, n = name, 1
            while name in names: name = name_ + str(n); n+=1
            axis._set(root=root, name=name)
            if force_at: root.axes[force_at] = axis
            else: root.axes.insert(0, axis)
        def at(axis, field):
            field = int(field) if this(field).inherits(str) and field.isdecimal() else field
            found = axis.index(field) if field in axis else None
            axis._to = found if found is not None else field
        def __repr__(_):
            return '{}: {}'.format(_.name, _._resize(' '.join([str(i) for i in _])))
    def __init__(this, **table_description):
        super().__init__(axes=this._axes())
        this.set(**table_description)
    def reset(data):
        base = None
        if len(data.axes)>0:
            base = [data.default]*len(data.axes[-1])
            for ax in reversed(data.axes[0:-1]): base = [base]*len(ax)
        data._set(data=array(base))
    @property
    def ax_names(_): return [ax.name for ax in _.axes]
    def at(data, axis):
        with this(axis) as _axis:
            if _axis.inherits(int):
                if axis>0 and axis<len(data.axes): return data.axes[axis]
            elif _axis.inherits(str):
                axes = data.ax_names
                if axis in axes: return data.axes[axes.index(axis)]
        return None
    def _check(build):
        if build.data is None: build.reset()
        return build.data
    def _find(_, inverted, ax_field):
        _._check()
        def index(axis, entry):
            fields = _.at(axis)
            if fields is not None:
                if this(entry).isiterable:
                    return tuple([fields.index(field) for field in entry])
                else: return ':'
            return None
        def translate(axis, found):
            if axis.name in found:
                _range = found[axis.name]
                if this(_range).inherits(tuple):
                    if inverted: found[axis.name] = tuple([field for field in range(len(axis)) if field not in _range])
                    return "_from['{}']".format(axis.name)
            return ':'
        found={field:index(field,entry) for field,entry in ax_field.items()}
        found={field:value for field,value in found.items() if value is not None}
        reshape='M['+','.join([translate(axis,found) for axis in _.axes])+']'
        _._set(_reshape_ = (reshape, found))
    def _by_val(_, depth=-1, _layer=0):
        M, axes = _._check(), _.axes
        do, _from = _.__dict__.pop('_reshape_') if '_reshape_' in _._meta() else (None, {})
        copy = super()._by_val(depth, _layer)
        copy.axes = []
        for ax in reversed(axes):
            fields = [field for n,field in enumerate(ax) if n in _from[ax.name]] if ax.name in _from else ax
            copy.set(**{ax.name:fields})
        copy.data = eval(do) if do else M.copy()
        return copy
    def _translate(_, directions):
        axes = directions.split(',')
        for ax_dir in axes:
            ax, field = [token.strip() for token in ax_dir.split(':')]
            axis = _.at(ax)
            if axis: axis.at(field)
        return '['+','.join([str(ax._to) for ax in _.axes])+']'
    def _get_set(_, directions, mode='get', value=None):
        if mode == 'get' and not '_MGET' in _.sets: _._MGET = []
        if len(directions) == len(_.axes):
            resolve, message = True, []
            for n,part in enumerate(directions):
                _part = this(part)
                if _part.inherits(int, str) or _part.isiterable and len(part)==1:
                    token = part if _part.inherits(str, int) else part[0]
                    message.append(':'.join([str(_.axes[n].name),str(part)]))
                else:
                    resolve = False
                    for token in part:
                        redirection = list(directions)
                        redirection[n] = token
                        _._get_set(tuple(redirection), mode, value)
            if resolve:
                message = ','.join(message)
                if mode=='get': _._MGET.append(_[message])
                else: _[message] = value
    def __getitem__(by, field_directions):
        M = by._check()
        if this(field_directions).inherits(tuple):
            by._get_set(field_directions)
            result = by.__dict__.pop('_MGET')
            return result
        else: return eval('M'+by._translate(field_directions))
    def __setitem__(by, field_directions, value):
        M = by._check()
        if this(field_directions).inherits(tuple): by._get_set(field_directions, 'set', value)
        else: exec('M'+by._translate(field_directions)+'=value')
    def set(data, **ax_field):
        for name, fields in ax_field.items(): data.axis(data, fields, name)
    def get(data, **ax_field):
        data._find(0, ax_field)
        return data._by_val()
    def let(data, **ax_field):
        data._find(1, ax_field)
        return data._by_val()
    @property
    def sets(tree): return set(meta(tree))
    def __repr__(self):
        M = self._check()
        _repr, dimensions = '', len(self.axes)
        if not dimensions: _repr += 'void table\n'
        else:
            dimensions = len(self.axes)
            y = self.axes[-2] if dimensions >= 2 else None
            if dimensions>2:
                y = self.axes[-2]
                for n,ax in enumerate(self.axes[:-2]): _repr += '{}{}: {}/{}\n'.format('\t'*n, ax.name, ax._to, len(ax))
            mr = eval('M'+str([ax.index(ax._to) for ax in self.axes][:-2])) if dimensions>2 else M
            pad = max([len(y.name)]+[len(str(field)) for field in y]+[len(str(value)) for line in mr for value in line])+self.PAD if dimensions>1 else 0
            _repr, x, spaces = _repr+y.name+'\n' if y else '', self.axes[-1], ' '*pad if pad>0 else '\t'
            header = spaces+''.join([str(field).ljust(pad) for field in x])
            _repr += self._resize(header) + '\n'
            ellipsis_at = self._lines_-self.ELLIPSIS_AT-1
            last_values_from = len(mr)-self.ELLIPSIS_AT
            if last_values_from<=ellipsis_at: last_values_from = ellipsis_at+1
            for n, line in enumerate(mr):
                if n<ellipsis_at or n>last_values_from:
                    values = str(y[n]).ljust(pad) if y else ''
                    values += ''.join([str(value).ljust(pad) for value in line])
                    _repr += self._resize(values) + '\n'
                elif n==ellipsis_at:
                    _repr += self._ellipsis_ + '\n'
        extra = {k:v for k,v in meta(self).items() if k != 'data' and k != 'axes'}
        _repr += self._resize(spaces*len(x)+x.name)+'\n'+'\n'.join(['{}:\t{}'.format(k, self._repr(v)) for k,v in extra.items()])
        return _repr
TAb = tab = Table

def set_repr_to(lines, ratio=.7):
    set_repr_to(lines)
    Table.ELLIPSIS_AT = int(Table._lines_*(1-ratio))

def butter_type(lowcut, highcut, fs, order=5, type='band'):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype=type)
    return b, a

def butter_filter(data, lowcut, highcut, fs, order=5, type='band'):
    b, a = butter_type(lowcut, highcut, fs, order=order, type=type)
    y = lfilter(b, a, data)
    return y

In [ ]:
!pip install pyEDFlib # installing the pyEDFlib library in the Google Colab environment

import pyedflib as edf

class EEG(Table):
    LABEL_START = 'EEG '
    BAD = ['TTL', 'ECG']
    BP_SEP = '-'
    class time:
        """ converts time units to seconds by frequency sampling (fs) """
        unit = 'units'
        def __init__(_, units): _.time = units
        def __call__(_, fs=None): return _.time/fs
        def __repr__(_): return '{} {}'.format(str(_.time),_.unit)
    class ms(time):
        """ converts ms to time units by frequency sampling (fs) """
        unit = 'ms'
        def __call__(_, fs=1000): return int(round(_.time*fs/1000))
    class secs(time):
        """ converts seconds to time units by frequency sampling (fs) """
        unit = 's'
        def __call__(_, fs=1000): return int(_.time*fs)
    def _load(eeg, epoch, n):
        data = None
        with edf.EdfReader(eeg.file) as file:
            data = [file.readSignal(eeg.labels[id], epoch.at, epoch.span) for id in eeg.labels]
            file.close()
        if data is not None:
            eeg._set(data=array(data), at_epoch=(n, epoch()))
        else:
            if 'at_epoch' in eeg.sets: del(eeg.at_epoch)
            eeg._set(data=None)
    class step(GSet):
        START = 0
        CENTRE = 1
        END = 2
        def __init__(step, space, duration):
            step._set(at=space, span=duration)
        def reset(grid, at=0, root=None):
            if root: grid._set(root=root)
            else: root = grid.root
            all_space, left = root.duration(root.fs), 0
            if grid.at.time == 0: epochs = [EEG.step(0, all_space)]
            else:
                space, span, epochs = grid.at(root.fs), grid.span(root.fs), []
                for x in range(at, all_space, space):
                    end = x+span
                    if end>all_space: left = all_space-x
                    else: epochs.append(EEG.step(x, span))
            grid._set(_all=epochs, skip=at, out=left)
        def __call__(step, _as=None):
            if 'root' in meta(step):
                if _as == None: return len(step._all)
            elif _as is not None: step.id = _as
            elif 'id' in meta(step): return step.id
        def items(wrapped):
            if 'root' in meta(wrapped): return wrapped._all
        def __getitem__(by, epoch_n):
            if 'root' in meta(by) and epoch_n<len(by._all):
                by.root._load(by._all[epoch_n], epoch_n)
        def __repr__(_): return '|'.join([repr(_.at),repr(_.span)])
    class event(GSet):
        def __init__(event, to=None, group=None, _as=0, _from=0):
            event._set(mode=_from, note=group, id=_as)
            if to is not None: event.link(to)
        def link(event, to):
            if event.note is None or not 'event' in to.sets:
                if event.note is None:
                    to.event = event
                    event.type = []
                    return
                else: EEG.event(to)
            types = to.event.type
            ids = [to.event.id]+[_type.id for _type in types]
            while event.id in ids: event.id += 1
            if event.note in to.notes:
                event.at = to.notes[event.note]
                types.append(event)
        def __repr__(event):
            _repr = str(event.id)
            if 'at' in meta(event): _repr += ' at: {}'.format(event.at)
            if 'type' in meta(event):
                for subev in event.type: _repr += '; '+repr(subev)
            return _repr
    def _at_last(eeg, sets):
        if 'epoch' in meta(eeg):
            eeg.epoch.reset(root=eeg)
            if len(eeg.axes.time) != eeg.epoch.span(eeg.fs): eeg.axis(eeg, eeg.epoch.span(eeg.fs), 'time', 1)
    @staticmethod
    def from_file(name, step=None, bad=None):
        def correct_(label):
            if label.startswith(EEG.LABEL_START): return label[len(EEG.LABEL_START):]
            return label
        eeg = EEG()
        with edf.EdfReader(name) as file:
            if bad is None: bad = EEG.BAD
            duration = EEG.secs(file.getFileDuration())
            fs = file.getSampleFrequencies()[0]
            if step is None: step = EEG.step(EEG.secs(0), duration)
            raw_notes = file.readAnnotations()
            notes = {note:[] for note in set(raw_notes[-1])}
            for n, note in enumerate(raw_notes[-1]):
                notes[note].append(EEG.secs(raw_notes[0][n]))
            labels = [correct_(label) for label in file.getSignalLabels()]
            labels = {label:n for n,label in enumerate(labels) if label not in bad}
            eeg.set(time=step.span(fs), region=tuple(labels))
            eeg(file=name, duration=duration, fs=fs, notes=notes, labels=labels, epoch=step)
            file.close()
        return eeg
    def remap(eeg, at=None, step=None):
        sets = eeg.sets
        if this(step).inherits(EEG.step): eeg._set(epoch=step)
        if at is None:
            at = eeg._best_map if '_best_map' in sets else 0
        eeg.epoch.reset(at)
        if 'event' in sets:
            deltas = []
            for epoch in eeg.epoch._all: epoch.id = None
            for event in eeg.event.type:
                for time in event.at:
                    at, space, limit = time(eeg.fs), eeg.epoch.at(eeg.fs), len(eeg.epoch.items())-1
                    for n,epoch in enumerate(eeg.epoch.items()):
                        end = epoch.at+space if n<limit else epoch.span
                        if at>=epoch.at and at<end:
                            epoch(event.id)
                            if event.mode == eeg.step.START: deltas.append(EEG.time(at-epoch.at)(eeg.fs))
                            elif event.mode == eeg.step.END: deltas.append(EEG.time(end-at-1)(eeg.fs))
                            else:
                                centre = epoch.at+int(round(epoch.span/2))
                                deltas.append(EEG.time(abs(at-centre))(eeg.fs))
                            break
            for epoch in eeg.epoch._all:
                if epoch() is None: epoch(eeg.event.id)
            eeg.deltas = deltas
    def optimize(eeg, *events, grid=None):
        if events:
            for event in events: event.link(eeg)
        eeg.remap(0, grid)
        gaussian_space = stats.shapiro if len(eeg.deltas)<=5000 else stats.normaltest
        def test():
            _, p = gaussian_space(eeg.deltas) if len(eeg.deltas)>2 else 0,1
            if p<=0.05: return p, median(eeg.deltas)
            return p, average(eeg.deltas)
        (p, best), at, check = test(), 0, eeg.epoch.span(eeg.fs)
        print('optimizing epoch position...', end=' ')
        for _try in range(1, check):
            eeg.remap(_try)
            p, check = test()
            if check<best: p, best, at = p, check, _try
        _test = 'median' if p<0.05 else 'mean'
        print('best frame found at {:.3f}s with a {} delay of {:.3f}s'.format(EEG.time(at)(eeg.fs), _test, EEG.time(best)(eeg.fs)))
        eeg._set(_best_map=at)
    class sampler(GSet, GRe):
        eeg = None
        def __init__(map, root, *reserve, **opts):
            raw, proc = [step() for step in root.epoch.items()], []
            find, key = None, {k:v for k,v in reserve}
            for step in raw:
                if find==step: find=None
                if find is None: proc.append(step)
                else: proc.append(None)
                if step in key: find = key[step]
            key = {k:[] for k in list(set(raw))+[None]}
            for n,id in enumerate(proc): key[id].append(n)
            map._set(eeg=root, key=key, mask=proc, **opts)
        def _at_last(_, sets):
            if 'seed' in sets: random.seed(_.seed)
        def set(map, **event_range):
            prev, key = map.key, {}
            for k,deltas in event_range.items():
                if k in prev:
                    seq, key[k] = prev[k], []
                    for item in seq: key[k] += [item+d for d in deltas]
                    for o in prev:
                        if o != k:
                            for e in key[k]:
                                if e in prev[o]: prev[o].pop(prev[o].index(e))
            for k in prev:
                if k not in key: key[k] = prev[k]
            map._set(prev=prev, key=key)
        def get(map, event, times, random_seed=None):
            if random_seed and not 'seed' in meta(map): map._set(seed=random_seed)
            if not 'pool' in meta(map): map._set(pool = {k:map.key[k].copy() for k in map.key})
            resampled, sequence = [], []
            while times:
                if len(map.pool[event])==0: map.pool[event] = map.key[event].copy()
                at = map.pool[event].pop(random.randint(len(map.pool[event])))
                map.eeg.epoch[at]
                resampled.append(map.eeg.data)
                sequence.append(at)
                times -= 1
            return resampled, sequence
        def __repr__(_): return _._resize('|'.join([str(id) if id!=None else ' ' for id in _.mask]))
    def tag(event, *a_b, **event_range):
        event._set(sample=event.sampler(event, *a_b))
        event.sample.set(**event_range)

STEp = epoch = EEG.step
TIME = EEG.time
SET = EEG.event
secs = EEG.secs
ms = EEG.ms

#### The cell below defines the preprocessing function for resampling and notch filtering the data.

In [ ]:
import numpy as np

def preprocess(eeg, epoch, limit=500):
    """Primary preprocessing. Resamples data to a limit frequency and applies a notch filter.

    Args:
        eeg (eeg): Wrapper object of the raw EEG data and metadata.
        epoch (list): Signal epoch.
        limit (int): Target frequency for resampling. Defaults to 500.

    Returns:
        list: Preprocessed epoch.
    """
    sampling, rse = limit, epoch
    if np.ceil(eeg.fs) == limit: rse = epoch

    elif np.ceil(eeg.fs)//limit == 2: rse = dwindle(epoch, int(np.ceil(eeg.fs)/limit)-1)
    elif np.ceil(eeg.fs)//limit == 4: rse = dwindle(epoch, int(np.ceil(eeg.fs)/limit)-3)

    elif np.ceil(eeg.fs)<limit: rse = upsample(epoch, np.ceil(eeg.fs), limit)
    else: print("Error: Cannot resample with the current version of preprocessing function.")

    nse = notch(rse, fs=sampling, order=2)
    
    return rse

#### The cell below implements six connectivity measures (seven, considering the double nature of spectral coherence - imaginary part and real part) to be computed between two nodes (signals):
###### 1) Phase-locking value
###### 2) Phase-lag index
###### 3) Spectral coherence
###### 4) Cross-correlation
###### 5) Phase-amplitude coupling
###### 6) Prediction error connectivity

In [ ]:
from numpy import correlate, average, array, angle, mean, sign, exp, zeros, abs, unwrap, fromfile, unpackbits, packbits
from scipy.signal import hilbert, csd

def phaselock(signal1, signal2):
    """Computes the phase locking value between two notch-filtered signals.

    Args:
        signal1 (array): Timecourse recorded from a first node.
        signal2 (array): Timecourse recorded from a second node.

    Returns:
        float: Phase locking value.
    """
    sig1_hil = hilbert(signal1)
    sig2_hil = hilbert(signal2)
    phase1 = angle(sig1_hil)
    phase2 = angle(sig2_hil)
    phase_dif = phase1-phase2
    plv = abs(mean(exp(complex(0,1)*phase_dif)))
    return plv

def phaselag(signal1, signal2):
    """Computes the phase lag index between two signals.

    Args:
        signal1 (array): Timecourse recorded from a first node.
        signal2 (array): Timecourse recorded from a second node.

    Returns:
        float: Phase lag index.
    """
    sig1_hil = hilbert(signal1)
    sig2_hil = hilbert(signal2)
    phase1 = angle(sig1_hil)
    phase2 = angle(sig2_hil)
    phase_dif = phase1-phase2
    pli = abs(mean(sign(phase_dif)))
    return pli

def spectral_coherence(signal1, signal2, fs, imag=False):
    """Computes the spectral coherence between two signals.

    Args:
        signal1 (array): Timecourse recorded from a first node.
        signal2 (array): Timecourse recorded from a second node.
        fs (int): Sampling frequency.
        imag (bool): If True, computed the imaginary part of spectral coherence, if False computes the real part. Defaults to False.

    Returns:
        float: Spectral coherence.
    """
    Pxy = csd(signal1,signal2,fs=fs, scaling='spectrum')[1]
    Pxx = csd(signal1,signal1,fs=fs, scaling='spectrum')[1]
    Pyy = csd(signal2,signal2,fs=fs, scaling='spectrum')[1]
    if imag: return average((Pxy.imag)**2/(Pxx*Pyy))
    elif not imag: return average(abs(Pxy)**2/(Pxx*Pyy))

def cross_correlation(signal1, signal2):
    """Computes the cross correlation between two signals.

    Args:
        signal1 (array): Timecourse recorded from a first node.
        signal2 (array): Timecourse recorded from a second node.

    Returns:
        float: Cross correlation.
    """
    return correlate(signal1, signal2, mode="valid")

def PAC(signal1, signal2, fs):
    """Computes low frequency phase-high frequency amplitude phase coupling between two signals.
    Low frequency = 1-4 Hz; High frequency = 30-70 Hz
    Args:
        signal1 (array): Timecourse recorded from a first node.
        signal2 (array): Timecourse recorded from a second node.
        fs (int): Sampling frequency.

    Returns:
        float: Phase-amplitude coupling.
    """
    low = butter_filter(signal1,1,4,fs)
    high = butter_filter(signal2,30,70,fs)
    low_hil = hilbert(low)
    low_phase_angle = unwrap(angle(low_hil))
    high_env_hil = hilbert(abs(hilbert(high)))
    high_phase_angle = unwrap(angle(high_env_hil))
    phase_dif = low_phase_angle - high_phase_angle
    plv = abs(mean(exp(complex(0,1)*phase_dif)))
    return plv

class bit:
    def __init__(my, size = 32): my.states = size
    def resize(this, bits):
        n, max = 0, 1
        for bit in bits:
            n += bit * max
            max <<= 1
        n = int(round(n / float(max) * this.states)) - 1
        max, bits = 1, []
        while(max < this.states):
            bits.append((n & max) / max)
            max <<= 1
        return bits

class AWC(struct, dict):
    train, limit = 10, 100
    bits = 8
    time = 8
    blur = None
    same = False
    class lnx:
        l, n = 1, 2
        @property
        def _clone(this):
            copy = AWC.lnx()
            copy.l, copy.n = this.l, this.n
            return copy
        def __add__(this, bit):
            bit, learn, limit = bit
            this.l += bit * learn
            this.n += learn
            if this.n > limit: this.n /= 2.; this.l /= 2.
        def __call__(set, weight): return set.l * weight, set.n * weight
    def __init__(context, **params):
        context._set(_codes=[], _last=None, **params)
    def clone(this, **changes):
        copy = AWC()
        copy.set(**this._clonable)
        copy.set(**changes)
        copy.update(this)
        for context in copy: copy[context] = copy[context]._clone
        return copy
    def _make(actual, set):
        actual._codes, context = [], []
        for bits in set:
            context += bits
            actual._codes.append(tuple(context))
    def __call__(actual):
        l, n, w = 1., 2., 1
        for length, context in enumerate(actual._codes):
            w *= len(context)
            if context in actual:
                _l, _n = actual[context](w)
                l += _l; n += _n
            else: actual[context] = AWC.lnx()
        return l/n
    def __add__(last, bit):
        for context in last._codes: last[context] + bit
    def learn(symbol, data=None, file=None, tell=False):
        if file: data = unpackbits(fromfile(file, dtype = 'ubyte'))
        check, to = None, 0.
        if tell: check = tell*len(data)
        train, limit = symbol.train, symbol.limit
        set, coded, time = [], [], symbol.time
        while time: set.append(list()); time -= 1
        for n, bit in enumerate(data.tolist()):
            if tell and n%check==0: print('{:.0%}|'.format(to), end=''); to+=tell
            if symbol.same: symbol._make([[n%symbol.bits]]+set[1:])
            else: symbol._make(set)
            coded.append(symbol())
            symbol + (bit, train, limit)
            base = set[0]
            if len(base) == symbol.bits:
                if symbol.blur: base = symbol.blur.resize(base)
                set.insert(1, packbits(base).tolist())
                set.pop(-1)
                set[0] = [bit]
            else: base.append(bit)
        symbol._last = data
        return dict(code=record(coded, **meta(data)), error=record(abs(coded-data), **meta(data)))

def error(matrix, layers=1, mmult=3, tell=.1, dtail=True):
    d = len(matrix)
    E, pairs = zeros((d,d)), []
    O, l = zeros((d,d)), zeros((d,d))
    for a in range(d):
        if dtail:
            for b in range(d): pairs.append((a,b))
        else:
            for b in range(a, d): pairs.append((a,b))
    to, check = None, None
    if tell: to, check = 0., int(tell*len(pairs))
    for n, (a, b) in enumerate(pairs):
        if tell=='all': print('.', end='')
        elif tell<1. and n%check==0: print('{:.0%}|'.format(to), end=''); to+=tell
        c, data = AWC(bits=2, time=2*mmult), create([matrix[a],matrix[b]])
        R = c.learn(binarize(data).serialized)
        if layers==1 or layers==3:
            E[b,a] += average(R['error'].deserialized[0])
            E[a,b] += average(R['error'].deserialized[1])
        if layers>1:
            O[a,b] = average(R['error'].deserialized[0])
            l[a,b] = average(R['error'].deserialized[1])
        del c,R
    if tell and tell!='all' and tell<1.: print()
    elif tell == 1.: print('.', end='')
    if layers==1: return record(E)
    elif layers==2: return record(O), record(l)
    return record(E), record(O), record(l)

def PEC(nse, n):
    """Computes prediction error connectivity.

    Args:
        nse (list): Preprocessed epoch (resampled and notched).
        n (int): Epoch index.

    Returns:
        array: Connectivity matrix.
    """
    print('{}: '.format(n), end='')
    return array(error(nse, 2)[1])


#### The function below computes the connectivity matrix, using one of the connectivity measures.

In [ ]:
from joblib import Parallel, delayed

def analyze_epoch(epoch, method, dtail, **opts):
    mat = zeros((len(epoch),len(epoch)))
    nid, pairs = list(range(len(epoch))), []
    for a in range(len(nid)):
        if dtail:
            for b in range(len(nid)): pairs.append((a,b))
        else:
            for b in range(a, len(nid)): pairs.append((a,b))

    parallelize = Parallel(n_jobs=-1)(delayed(method)(epoch[pair[0]], epoch[pair[1]], **opts) for pair in pairs)
    conn_per_pair = [p for p in parallelize]

    for pair_idx, pair in enumerate(pairs):
        mat[pair[0],pair[1]] = conn_per_pair[pair_idx]
    return mat

def connectivity_analysis(epochs, method, dtail=False, **opts):
    """Computes a connectivity matrix N×N (N - number of nodes) per epoch, containing connectivity method measures for all node pairs.

    Args:
        epochs (list): List of preprocessed epochs (resampled, filtered and notched).
        method (function): Connectivity method.
        dtail (bool): If True, computes a square matrix; if False, computes a tringular matrix. Defaults to False.
        opts (optional): method-specific arguments.
    Returns:
        list: List of connectivity matrices for all epochs.
    """
    print('Connectivity measure: '+str(method).split()[1])
    parallelize = Parallel(n_jobs=-1)(delayed(analyze_epoch)(e, method, dtail, **opts) for e in epochs)
    cm = [mat for mat in parallelize]
    return cm

#### Now that we have defined the objects for loading, handling and processing the data, we proceed with the experiment.

#### Define the path to your main folder with data in your Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

main_folder = "/content/gdrive/My Drive/Ext-val/"

#### The user will now be asked to choose from a number of parameters (see *Guidelines*).
###### Choose from the available options by typing the number assigned to the option.

In [ ]:
woi = "30sec"

method_idx = input("Connectivity method:\n 1. PEC\n 2. Spectral Coherence\n 3. Phase Lock Value\n 4. Phase-Lag Index\n 5. Cross-correlation\n 6. Phase-amplitude coupling\n Indicate a number: ")

ext = ""
if "2" == method_idx:
    im = input("Imaginary part (Y/N): ").upper()
    if im == "Y": imag,ext = True,"I"
    elif im == "N": imag,ext = False,"R"

bands, Bands = input("Filter the signal: Y/N ").upper(), False

if bands=="N": bands = "w"
elif bands=="Y":
    Bands = True
    mn = int(input("Set band range min: "))
    mx = int(input("Set band range max: "))
    bands = (mn,mx)

method_code = {'1':"PEC", '2':"SC", '3':"PLV", '4':"PLI", '5':"CC", '6':"PAC"}

#### The next cell defines the path to the EDF file.
###### Filename should be formatted as: patientacronym*-baseline.EDF or patientacronym*-seizure.EDF (asterix signifies a regular expression for any string of choice).
###### Subject identifier is saved as a three-letter acronym.

In [ ]:
import os

file_seizure = main_folder +  "Ext-val-edf/sub-jh101_run-01.edf"

subject_id = os.path.basename(file_seizure).split('_')[0]
print("Subject ID:", subject_id)

run_id = os.path.basename(file_seizure).split('_')[-1].split('.')[0]
print("Run ID:", run_id)

#### Connectivity is measured in epochs of data. The cell below loads the EDF files and defines the epochs.
###### *(These epochs will be classified by the support vector machine (a machine-learning classifier) between baseline and a window of interest (WOI) in the gcolab_find_en.ipynb notebook.)*
###### The WOI is fragmented into epochs of 1 second (*span*), with half-second overlaps (*step*).
###### To keep the dataset balanced for the classification, we considered a fixed number of epochs, irregardless of the WOI duration. This number was fixed to half of the all epochs in a 1-minute window (119), using the pre-set *span* and *step* parameters.
###### The raw data is loaded as *EEG* class object, with the defined epoch parameters.

In [ ]:
# Define step and span for a 30-s window, to have 119 epochs in total
span, step = 1000, 250      # in ms
min_woi_duration = 30000    # in ms
n_epochs = int((min_woi_duration/step)-1)

print("Number of epochs to consider for classification =", n_epochs)

eeg_seizure = EEG.from_file(file_seizure, epoch(ms(step), ms(span)))    # load raw seizure SEEG data as an EEG object (class)

#### Import the clinical annotations from *onset_annotations.xlsx*.

In [ ]:
import pandas as pd

def find_annotations(sub_id, run_id):
    """
    Find the seizure onset and offset annotations from the Excel file.

    Parameters:
    - sub_id (str): Subject ID.
    - run_id (str): Run ID.

    Returns:
    - onset (float): Seizure onset time (s)
    - offset (float): Seizure offset time (s)
    """
    onset, offset = None, None

    # Load the DataFrame from the Excel file
    df = pd.read_excel("/content/gdrive/My Drive/Ext-val/onset_annotations.xlsx")

    # # Discard rows with a value of 1 in the 'UNCLEAR' column
    # df = df[df['UNCLEAR'] != 1] # subjh106 unique run is unclear

    # Find the row with the specified SUB_ID and RUN_ID
    selected_row = df[(df['SUB_ID'] == sub_id) & (df['RUN_ID'] == run_id)]

    # Check if the row is found
    if not selected_row.empty:
        # Extract 'ONSET_TIME' and 'OFFSET_TIME' from the selected row
        onset = selected_row['ONSET_TIME'].values[0]
        offset = selected_row['OFFSET_TIME'].values[0]

    return onset, offset


In [ ]:
# Find seizure onset and offset times in the manually created Excel table
seizure_onset, seizure_offset = find_annotations(subject_id, run_id)

sz_start_note, sz_end_note = 'Start', 'End'

eeg_seizure.notes[sz_start_note] = [secs(seizure_onset)]
eeg_seizure.notes[sz_end_note] = [secs(seizure_offset)]

#### Check file duration. If it's inconsistent with the metadata from *metadata.xlsx*, set it manually.

In [ ]:
def get_duration(sub_id, run_id):
    """
    Find the seizure onset and offset annotations from the Excel file.

    Parameters:
    - sub_id (str): Subject ID.
    - run_id (str): Run ID.

    Returns:
    - duration (float): Recording duration (s)
    """

    # Load the DataFrame from the Excel file
    df = pd.read_excel("/content/gdrive/My Drive/Ext-val/metadata.xlsx")

    # Find the row with the specified SUB_ID and RUN_ID
    selected_row = df[(df['SUB_ID'] == sub_id) & (df['RUN_ID'] == run_id)]

    # Check if the row is found
    duration = selected_row['RECORDING_LEN'].values[0] if not selected_row.empty else None

    return duration

In [ ]:
if eeg_seizure.duration != get_duration(subject_id, run_id):
    eeg_seizure.duration = secs(get_duration(subject_id, run_id))
     
print("Seizure file duration:", eeg_seizure.duration)

#### Check channel labels and define bad channels.

In [ ]:
import os
import pandas as pd
import numpy as np

def find_bad_channels(ieeg_dir):
    """
    For each ieeg file, find bad channels by checking the corresponding channels.tsv file.

    Parameters:
    - ieeg_dir (str): Path to the directory containing ieeg files.

    Returns:
    - bad_channels_dict (dict): Dictionary with ieeg filenames as keys and lists of bad channels as values.
    """
    bad_channels_dict = {}

    for ieeg_file_name in os.listdir(ieeg_dir):
        if ieeg_file_name.endswith(".eeg"):

            # Extract sub_id and run_id from the ieeg filename
            sub_id = ieeg_file_name.split('_')[0]
            run_id = ieeg_file_name.split('_')[-2]
            # print(sub_id, run_id)

            # Search for all channels.tsv files in the channels_tsv_dir
            for channels_tsv_name in os.listdir(ieeg_dir):
                if channels_tsv_name.endswith("_channels.tsv") and sub_id in channels_tsv_name and run_id in channels_tsv_name:

                    channels_tsv_path = os.path.join(ieeg_dir, channels_tsv_name)

                    # Load channel information from channels.tsv
                    channels_df = pd.read_csv(channels_tsv_path, delimiter='\t')

                    # Extract bad channels
                    bad_channels = channels_df[channels_df['status'] == 'bad']['name'].tolist()

                    # Store bad channels in the dictionary
                    bad_channels_dict[f"{sub_id}_{run_id}"] = bad_channels
                    break  # Break the loop once matching channels_tsv is found

    return bad_channels_dict

def exclude_bads(epoch, bad_channels_ids):
    """
    Exclude bad channels from an epoch.

    Parameters:
    - epoch (numpy.ndarray): The original epoch data, assumed to be a 2D NumPy array.
    - bad_channels_ids (list): List of channel indices to be excluded.

    Returns:
    - epoch_without_bads (numpy.ndarray): The epoch data after excluding bad channels.
    """
    # Create a boolean mask for the rows to exclude
    mask = np.ones(epoch.shape[0], dtype=bool)
    mask[bad_channels_ids] = False

    # Exclude the rows based on the boolean mask
    epoch_without_bads = epoch[mask]

    return epoch_without_bads

def exclude_node_from_cm(cm_list, channel_id):
    """Excludes a node from connectivity matrices of cm_list epochs.
    Arguments:
        cm_list (list of numpy arrays): list of connectivity matrices saved in the PREP file under X;
                                        First half of the matrices are from seizure epochs, second half form baseline epochs.
        channel_id (int or list): index of the node to be removed
    Returns the updated list of matrices.
    """
    reduced_cms = []
    for cm in cm_list:
      cm = np.delete(cm, channel_id, 0)
      cm = np.delete(cm, channel_id, 1)
      reduced_cms.append(cm)
    return reduced_cms

In [ ]:
nodes = list(eeg_seizure.axes.region)

# Extract bad channels for the current file
bad_channels_result = find_bad_channels(main_folder + subject_id + "/ses-presurgery/ieeg/")
bad_channels = bad_channels_result.get(f"{subject_id}_{run_id}", [])

# # Exclude bad channels
# # Remember that this should not be done if you want to use the exclude_node_form_cm function later,
# # as you will need the channel index reference
# for ch in bad_channels: nodes.remove(ch); print(f"Removed bad channel {ch}")

#### Check the sampling frequency.
###### The code below defines the frequency to which data will be resampled to (500 Hz), to homogenize across files.

In [ ]:
print("Sampling freqency (seizure file) =", eeg_seizure.fs)

# set the desired resampled frequency to 500 Hz if sampling is not already 512 Hz
resampling = 500

#### The following cell labels the baseline and seizure epochs relative to the position of clinical annotations (see *Guidelines*).
###### WOI is always defined relative to the *sz_start_note*.

In [ ]:
SET(eeg_seizure, _as='N')                      # N - baseline (non-seizure)
SET(eeg_seizure, sz_start_note, 'W')            # W - WOI
SET(eeg_seizure, sz_end_note, 'S', epoch.END)    # S - seizure

print(eeg_seizure.event)

#### Next, we optimize the positions of epochs relative to the clinical annotations.

In [ ]:
eeg_seizure.optimize()
eeg_seizure.remap()

#### The seizure duration is saved in the *units* variable, as the number of epochs present between the clinical annotations of seizure start and end.

In [ ]:
units = int((eeg_seizure.notes[sz_end_note][0].time - eeg_seizure.notes[sz_start_note][0].time)*(span/step))
print("Seizure length =", units, f"({units/(span/step)} s)")

#### Next, we introduce a virtual mask into the *EEG* class object, which indicates the time window to which an epoch belongs to (baseline, WOI or seizure).

In [ ]:
eeg_seizure.tag(('W', 'S'), W=range(-units-n_epochs,-units,1), S=range(-units+n_epochs,-units,-1))

#### Fetch the WOI (*a*) and baseline (*b*) epochs, their indices and labels.
###### The epoch labels are set as 0 for seizure and 1 for baseline. These labels will be referred to as ground truth by the classifier.

In [ ]:
a, ai = eeg_seizure.sample.get('W', n_epochs)   # fetch epochs and epoch indices
b, bi = eeg_seizure.sample.get('S', n_epochs)
i = ai + bi                                     # save indices
x = a + b                                       # save epochs
y = [1]*n_epochs + [0]*n_epochs                 # save epoch labels (1 for seizure, 0 for baseline)

print("Total number of epochs (seizure + baseline) =", len(x))
print("Epoch shape =", a[0].shape)

import matplotlib.pyplot as plt

j = random.randint(0, n_epochs-1)
plt.figure(figsize=(3,3))
plt.plot(a[j][1])
plt.title("Epoch example (seizure file)")
plt.show()

#### Exclude bad channels from epochs' arrays.

In [ ]:
bad_channels_ids = [nodes.index(node) for node in bad_channels]

clean_x = [exclude_bads(epoch, bad_channels_ids) for epoch in x]
x = clean_x

#### Resample and notch-filter the data.

In [ ]:
pp_seizure = [preprocess(eeg_seizure, ep, resampling) for i,ep in enumerate(a)]
print("Resampled to", pp_seizure[0].shape)

#### Filter the data in the set frequency band, if defined.

In [ ]:
fpp_seizure = [band(e, bands, pp_seizure[0].shape[1]) for e in pp_seizure] if Bands else pp_seizure

#### Compute the connectivity matrices for all epochs.
###### The matrices are saved in the *struct* class object, which serves as a table and facilitates accessing the results.

In [ ]:
import numpy as np

cm = struct(x=np.array(x), y=np.array(y), i=np.array(i)) # initiating an object for storing a connecivity matrix with shape (x, y) and epoch indices

cm._set(nodes = nodes)

epochs = fpp_seizure

if method_code[method_idx] == "SC_":
    cm._set(X = connectivity_analysis(epochs, spectral_coherence, fs=resampling, imag=imag))

elif method_code[method_idx] == "PEC":
    parallelize = Parallel(n_jobs=-1)(delayed(PEC)(ep,i+1) for i,ep in enumerate(fpp_seizure))
    cm_pec = [p for p in parallelize]
    cm._set(X = cm_pec)

elif method_code[method_idx] in "PLV":
    cm._set(X = connectivity_analysis(epochs, phaselock))

elif method_code[method_idx] == "PLI":
    cm._set(X = connectivity_analysis(epochs, phaselag))

elif method_code[method_idx] == "CC":
    cm._set(X = connectivity_analysis(epochs, cross_correlation))

elif method_code[method_idx] == "PAC":
    cm._set(X = connectivity_analysis(epochs, PAC, dtail=True, fs=resampling))


#### Check if the matrix row number corresponds to the number of good channels.

In [ ]:
cm.X[0].shape[0] == len(nodes)-len(bad_channels_ids)

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(cm.X[-1], cmap='Blues', interpolation='nearest')
plt.title("Connectivity matrix example")
plt.show()
print(cm.X[-1])

print("Seizure CM shape =", cm.X[0].shape)
print("Baseline CM shape =", cm.X[-1].shape)
print("Number of CMs saved =", len(cm.X))

#### The connectivity matrices are saved in a subfolder called *connectivity_matrices*, as files with a custom extension (*PREP*).
###### The filenames indicate the preprocessing parameters, as:
###### "*SUB-preseizure5-CC-(70,180).prep*" if the signal was filtered, and
###### "*SUB-preseizure5-PEC.prep*" if the signal was not filtered.

In [ ]:
# path_cm = main_folder + "connectivity_matrices/"
# # Create output directory if it doesn't exist
# os.makedirs(path_cm, exist_ok=True)

# if Bands:          REc(cm).save(path_cm + f"{subject_id}-{woi_code[woi]}-{method_code[method_idx]}{ext}-{bands}.prep".replace(" ",""))
# elif not Bands:    REc(cm).save(path_cm + f"{subject_id}-{woi_code[woi]}-{method_code[method_idx]}{ext}.prep")